In [4]:
import pandas as pd
data = pd.read_csv("../Website/data/complete.csv", on_bad_lines='skip')
data['datetime'] = pd.to_datetime(data['datetime'], errors='coerce')
df=data[data["country"]=="us"].sort_index(level="datetime")
df = df.dropna(subset=['datetime'])
df = df.sort_values(by='datetime').reset_index(drop=True)
df=df.dropna()

C:\Users\anton\AppData\Local\Temp\ipykernel_102368\2719364312.py:2: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../Website/data/complete.csv", on_bad_lines='skip')


In [10]:
#import pandas_profiling

In [6]:
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, Span
from bokeh.io import output_notebook

# output_notebook()  # Uncomment if using Jupyter Notebook

# === Load & Clean Data ===
data = pd.read_csv("../Website/data/complete.csv", on_bad_lines='skip')
data['datetime'] = pd.to_datetime(data['datetime'], errors='coerce')
df = data[data["country"] == "us"].dropna(subset=['datetime'])
df['year'] = df['datetime'].dt.year
year_counts = df['year'].value_counts().sort_index()

sightings_source = ColumnDataSource(data=dict(
    year=year_counts.index.tolist(),
    count=year_counts.values.tolist()
))

# === Annotated Media Events ===
alien_media = {
    1951: ("The Day the Earth Stood Still", "movie"),
    1977: ("Close Encounters", "movie"),
    1979: ("Alien", "movie"),
    1982: ("E.T.", "movie"),
    1986: ("Aliens", "movie"),
    1993: ("The X-Files (TV)", "tv"),
    1996: ("Independence Day", "movie"),
    1998: ("X-Files: The Movie", "tv"),
    2002: ("Signs", "movie"),
    2009: ("District 9", "movie"),
    2014: ("Edge of Tomorrow", "movie")
}

media_data = {
    'year': [],
    'count': [],
    'title': [],
    'type': []
}
for year, (title, typ) in alien_media.items():
    if year in year_counts.index:
        media_data['year'].append(year)
        media_data['count'].append(year_counts[year])
        media_data['title'].append(title)
        media_data['type'].append(typ)

media_source = ColumnDataSource(media_data)

# === Create Bokeh Plot ===
p = figure(
    width=1000, height=500,
    background_fill_color="#0f0f1f", border_fill_color="#0f0f1f",
    title="U.S. UFO Sightings with Major Alien Media Releases",
    x_axis_label=None, y_axis_label=None,
    x_range=(min(year_counts.index)-1, max(year_counts.index)+1),
    tools="", toolbar_location=None
)

p.vbar(x='year', top='count', width=0.8, source=sightings_source,
       color="#00ff99", line_color=None, legend_label="Sightings")

# Neon markers for media events
media_renderer = p.circle(x='year', y='count', size=10,
                          color="#ff3366", alpha=0.9, source=media_source,
                          legend_label="Media Event")

# Tooltip for media events
p.add_tools(HoverTool(
    renderers=[media_renderer],
    tooltips=[("Media", "@title"), ("Year", "@year"), ("Sightings", "@count")],
    mode='mouse'
))

# Styling polish
p.xaxis.major_label_text_color = "#cccccc"
p.yaxis.major_label_text_color = "#cccccc"
p.xaxis.axis_line_color = "#333333"
p.yaxis.axis_line_color = "#333333"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = "#222244"
p.title.text_color = "#eeeeee"
p.title.text_font_size = "16pt"
p.title.align = "center"
p.legend.label_text_color = "#cccccc"
p.legend.background_fill_alpha = 0.0
p.legend.location = "top_left"

# Show it
show(p)


C:\Users\anton\AppData\Local\Temp\ipykernel_102368\3369184138.py:9: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../Website/data/complete.csv", on_bad_lines='skip')


In [7]:
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, LabelSet
from bokeh.palettes import Category20

# === Load and preprocess data ===
data = pd.read_csv("../Website/data/complete.csv", on_bad_lines='skip')
data["datetime"] = pd.to_datetime(data["datetime"], errors="coerce")
df = data[data["country"] == "us"].dropna(subset=["datetime", "shape"])
df["year"] = df["datetime"].dt.year

# === Compute total sightings per year (all shapes) ===
totals = df.groupby("year").size()
totals = totals[totals.index >= 1960]

# === Full shape distribution (pivot) ===
shape_counts_all = df.groupby(["year", "shape"]).size().unstack(fill_value=0)

# === Select top 5 shapes, group rest as 'uncommon shapes' ===
shape_totals = shape_counts_all.sum().sort_values(ascending=False)
top_shapes = shape_totals.head(5).index
other_shapes = shape_totals.index.difference(top_shapes)

# Create new DataFrame for top 5 + 'uncommon shapes'
shape_counts = shape_counts_all[top_shapes].copy()
shape_counts["uncommon shapes"] = shape_counts_all[other_shapes].sum(axis=1)
shape_counts = shape_counts[shape_counts.index >= 1960]

# === Stack plot source ===
final_shapes = ["uncommon shapes"] + list(top_shapes)  # Move to bottom
source_data = {"year": shape_counts.index.tolist()}
colors = Category20[20][:len(final_shapes)]
for s in final_shapes:
    source_data[s] = shape_counts[s].values
source = ColumnDataSource(source_data)

# === Emoji map ===
emoji_map = {
    "light": "💡", "triangle": "🔺", "circle": "⭕",
    "fireball": "🔥", "unknown": "❓", "uncommon shapes": "🛸"
}

# === Media events ===
media = {
    1951: "The Day the Earth Stood Still",
    1977: "Close Encounters", 1979: "Alien", 1982: "E.T.",
    1986: "Aliens", 1993: "The X-Files (TV)", 1996: "Independence Day",
    1998: "X-Files: The Movie", 2002: "Signs",
    2009: "District 9", 2014: "Edge of Tomorrow"
}
media_years = [y for y in media if y in totals.index]
media_source = ColumnDataSource(data=dict(
    year=media_years,
    count=[totals[y] for y in media_years],
    title=[media[y] for y in media_years]
))

# === Bokeh figure ===
xmin, xmax = shape_counts.index.min(), shape_counts.index.max()
p = figure(
    width=1000, height=500,
    x_range=(1960, xmax + 3),
    y_range=(0, shape_counts.sum(axis=1).max() * 1.15),
    background_fill_color="#0f0f1f",
    title="U.S. UFO Shape Distribution Over Time (All Shapes)",
    tools="", toolbar_location=None
)

# === Area stack ===
p.varea_stack(
    stackers=final_shapes,
    x="year",
    color=colors,
    legend_label=[f"{emoji_map.get(s, '🛸')} {s}" for s in final_shapes],
    source=source
)

# Then reverse the legend manually (Bokeh workaround)
p.legend.items = list(reversed(p.legend.items))




# === Media dots ===
media_renderer = p.circle(
    'year', 'count',
    source=media_source,
    size=12,
    fill_color="#ffd700",
    line_color="white",
    line_width=1.3,
    legend_label="Media Event",
    level="overlay"
)
p.add_tools(HoverTool(
    renderers=[media_renderer],
    tooltips=[("Movie", "@title"), ("Year", "@year"), ("Sightings", "@count")],
    mode="mouse", point_policy="follow_mouse"
))

# === Emojis ===
recent = shape_counts.tail(25)
max_h = recent.sum(axis=1).max()
x_pts, y_pts, emojis, sizes = [], [], [], []
for i, s in enumerate(final_shapes):
    yr = recent[s].idxmax()
    below = recent[final_shapes[:i]].loc[yr].sum()
    band = recent[s].loc[yr]
    x_pts.append(yr)
    y_pts.append(below + band / 2)
    emojis.append(emoji_map.get(s, "🛸"))
    sizes.append(f"{10 + band / max_h * 16 - 4:.0f}pt")

emoji_source = ColumnDataSource(dict(x=x_pts, y=y_pts, emoji=emojis, size=sizes))
p.add_layout(LabelSet(
    x='x', y='y', text='emoji', source=emoji_source,
    text_font_size='size', text_align="center", text_baseline="middle"
))

# === Style ===
p.xaxis.axis_label = "Year"
p.yaxis.axis_label = "Sightings"
p.xaxis.major_label_text_color = "#cccccc"
p.yaxis.major_label_text_color = "#cccccc"
p.xaxis.axis_line_color = "#333333"
p.yaxis.axis_line_color = "#333333"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = "#222244"
p.title.text_color = "#eeeeee"
p.title.text_font_size = "16pt"
p.legend.label_text_color = "#ffffff"
p.legend.background_fill_alpha = 0
p.legend.location = "top_left"
p.legend.label_text_font_size = "9pt"

show(p)


C:\Users\anton\AppData\Local\Temp\ipykernel_102368\2029950978.py:7: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../Website/data/complete.csv", on_bad_lines='skip')
